In [1]:
using Evolutionary
using Statistics
using Random

In [2]:
function rk4(f, s0, x0, xf, n, a, b)
    x = range(x0, stop = xf, length = n+1)  # x grid
    s = repeat([s0], n+1, 1)  # array of the state of the system for each x
    h = x[2] - x[1]  # step size
    for i in 1:n
        k0 = h * f(s[i], a, b)
        k1 = h * f(s[i] + 0.5 * k0, a, b)
        k2 = h * f(s[i] + 0.5 * k1, a, b)
        k3 = h * f(s[i] + k2, a, b)
        s[i+1] = s[i] + (k0 + 2*(k1 + k2) + k3) / 6
    end
    return x, s
end

rk4 (generic function with 1 method)

In [3]:
function f(u, a, b)
    S, I, R = u
    N = sum(u)
    dS = -a*S*I/N  # dS/dt = -aS(t)I(t)
    dI = a*S*I/N - b*I  # dI/dt = aS(t)I(t) - bI(t)
    dR = b*I  # dR/dt = bI(t)
    return [dS, dI, dR]
end

f (generic function with 1 method)

In [4]:
function MAPE(xs, ys) #average absolute error
    return mean(abs.(xs .- ys)./xs)
end

MAPE (generic function with 1 method)

In [5]:
beta = 0.2
gamma = 0.1
x, data = rk4(f, [235.0, 14.0, 0.0], 0., 109., 1090, beta, gamma)
I = [x[2] for x in data];

In [6]:
function fitness_func(solution) #solution = [beta, gamma]
    x, s = rk4(f, [235.0, 14.0, 0.0], 0., 109., 1090, solution[1], solution[2])
    I_m = [x[2] for x in s]
    return MAPE(I, I_m) #albo MAPE(I_m, I)
end

fitness_func (generic function with 1 method)

In [8]:
Evolutionary.optimize(
                fitness_func, 
                BoxConstraints([0.0, 0.0], [1.0, 1.0]),
                rand(2),
                GA(populationSize = 100, selection = susinv, crossover = AX, mutation = uniform(0.1), mutationRate = 0.5, ε = 0.1)
)


 * Status: success

 * Candidate solution
    Minimizer:  [0.20031902155640857, 0.09978247184443811]
    Minimum:    0.003024795224118891
    Iterations: 108

 * Found with
    Algorithm: GA[P=100,x=0.8,μ=0.5,ɛ=0.1]

 * Convergence measures
    |f(x) - f(x')| = 0.0 ≤ 1.0e-12

 * Work counters
    Seconds run:   12.918 (vs limit Inf)
    Iterations:    108
    f(x) calls:    10900
